In [1]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-23 06:36:32--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv.1’

customer_churn.csv. 100%[===================>] 668.81K  --.-KB/s    in 0.05s   

2019-04-23 06:36:32 (12.6 MB/s) - ‘customer_churn.csv.1’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("customer_churn.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
!pip install -q tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 391kB/s 
    100% |████████████████████████████████| 3.0MB 8.1MB/s 
    100% |████████████████████████████████| 419kB 19.4MB/s 
    100% |████████████████████████████████| 61kB 26.2MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [7]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [0]:
numeric_columns=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
categorical_columns=['Geography','Gender','HasCrCard','IsActiveMember']

In [0]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)
train, valid = train_test_split(train, test_size=0.2, random_state=1)

In [0]:
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [0]:
feature_columns = []

In [0]:
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

In [16]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
def get_one_hot_rom_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(
  colname,
  train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_rom_categorical(col))

In [19]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(c

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer=layers.DenseFeatures(feature_columns)

In [22]:
feature_layer

In [0]:
model = keras.Sequential()
model.add(feature_layer)
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [27]:
model.fit(train_ds,
         validation_data = valid_ds,
         epochs=5)

W0423 06:37:05.190444 140338336847744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0423 06:37:05.226197 140338336847744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0423 06:37:05.233118 140338336847744 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please

Epoch 1/5
200/200 [==============================] - 4s 20ms/step - loss: 0.4115 - accuracy: 0.8082 - val_loss: 0.3852 - val_accuracy: 0.8363
Epoch 2/5
200/200 [==============================] - 4s 18ms/step - loss: 0.3474 - accuracy: 0.8565 - val_loss: 0.3785 - val_accuracy: 0.8406
Epoch 3/5
200/200 [==============================] - 4s 18ms/step - loss: 0.3371 - accuracy: 0.8623 - val_loss: 0.3753 - val_accuracy: 0.8438
Epoch 4/5
200/200 [==============================] - 4s 18ms/step - loss: 0.3320 - accuracy: 0.8633 - val_loss: 0.3742 - val_accuracy: 0.8438
Epoch 5/5
200/200 [==============================] - 4s 18ms/step - loss: 0.3282 - accuracy: 0.8628 - val_loss: 0.3733 - val_accuracy: 0.8431


In [39]:
print("Model Evaluate: ", model.evaluate(test_ds))

print("Train Exited value counts: ", train.Exited.value_counts())

print("Test Exited value counts: ", test.Exited.value_counts())

63/63 [==============================] - 1s 12ms/step - loss: 0.3388 - accuracy: 0.8660
Model Evaluate:  [0.3387696042893425, 0.866]
Train Exited value counts:  0    5108
1    1292
Name: Exited, dtype: int64
Test Exited value counts:  0    1585
1     415
Name: Exited, dtype: int64


In [0]:
preds=model.predict(test_ds)

In [0]:
import numpy as np

In [33]:
labels = (preds > 0.5).astype(np.int)
print(labels)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [0]:
from sklearn import metrics

In [35]:
ytest= test.Exited.values
print(metrics.classification_report(ytest, labels))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92      1585
           1       0.72      0.57      0.64       415

   micro avg       0.87      0.87      0.87      2000
   macro avg       0.81      0.76      0.78      2000
weighted avg       0.86      0.87      0.86      2000



In [0]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(ytest, labels)

array([[1495,   90],
       [ 178,  237]])